# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [1]:
# your answer here
import pandas as pd
import numpy as np
import math
import statistics

#### Open the ` Employee.csv` file and name your dataset `employee`.

In [2]:
# your answer here
employee = pd.read_csv('data/Employee.csv')
employee

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [3]:
# Method 1
# your answer here
mean = employee.groupby('Department').mean()
display(mean)


,Years,Salary
Department,,
HR,4.666667,45.00
IT,4.500000,48.75
Sales,2.500000,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [4]:
# Method 2
# your answer here
mean1 = employee.pivot_table(index='Department',values=['Salary'])
display(mean1)

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


In [5]:
# your answer here
mean2 = employee.pivot_table(index=['Department', 'Title'],values=['Salary'])
display(mean2)

Salary
Department Title            
HR         VP           70.0
           analyst      32.5
IT         VP           70.0
           analyst      32.5
           associate    60.0
Sales      associate    55.0

#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [6]:
# your answer here
years = employee.pivot_table(index='Title',values=['Salary','Years'])
display(years)

,Salary,Years
Title,,
VP,70.000000,8.00
analyst,32.500000,2.25
associate,56.666667,4.00


#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [7]:
# your answer here
information = employee.pivot_table(index='Title',values=['Salary','Department'], aggfunc = {'Salary':'mean','Department': 'count'})
display(information)

,Department,Salary
Title,,
VP,2,70.000000
analyst,4,32.500000
associate,3,56.666667


#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [8]:
# your answer here
data = employee.pivot_table(index=['Department', 'Title'],values=['Salary'], aggfunc = {'median','max','min','std'})
display(data)
# The NaN values are in std because in the formula n = 1 and it's impossible to divide with 0. 
#If there is only one person there isn't std because mean is equal to the olny value we have 

Salary                       
                        max median   min       std
Department Title                                  
HR         VP          70.0   70.0  70.0       NaN
           analyst     35.0   32.5  30.0  3.535534
IT         VP          70.0   70.0  70.0       NaN
           analyst     35.0   32.5  30.0  3.535534
           associate   60.0   60.0  60.0       NaN
Sales      associate   55.0   55.0  55.0  0.000000

#### Based on your comments, fill the missing values with an appropriate value.

In [9]:
# your answer here
data = data.replace(np.nan,0)
display(data)

Salary                       
                        max median   min       std
Department Title                                  
HR         VP          70.0   70.0  70.0  0.000000
           analyst     35.0   32.5  30.0  3.535534
IT         VP          70.0   70.0  70.0  0.000000
           analyst     35.0   32.5  30.0  3.535534
           associate   60.0   60.0  60.0  0.000000
Sales      associate   55.0   55.0  55.0  0.000000

#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [10]:
# your answer here
#more_information = employee.pivot_table(index=['Department'],values=['Salary','Department'], aggfunc = {'Salary':'sum','Department': 'count'})
#display(more_information)

#We can't use the same column for index and values, so we can't, but we can improve it
employee_copy = employee.copy()
employee_copy['Department1'] = employee_copy['Department']
more_information = employee_copy.pivot_table(index=['Department'],values=['Salary','Department1'], aggfunc = {'Salary':'sum','Department1': 'count'})
display(more_information)

,Department1,Salary
Department,,
HR,3,135
IT,4,195
Sales,2,110


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [11]:
# your answer here
total_years = employee.pivot_table(index=['Department','Title'],values=['Salary','Years'], aggfunc = {'Salary':'mean','Years': 'sum'})
display(total_years)

Salary  Years
Department Title                   
HR         VP           70.0      8
           analyst      32.5      6
IT         VP           70.0      8
           analyst      32.5      3
           associate    60.0      7
Sales      associate    55.0      5

### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [12]:
# your answer here
copy = employee.copy()
def delete_biggests_salaries(row):
    departments = row['Department'].unique().tolist()
    departments = ['0'] + departments
    b = 0
    max_salaries = []
    for name in departments:
        b = 0
        for i in range(len(row['Department'])):
            if (name == row['Department'][i]) & (b < row['Salary'][i]):
                b = row['Salary'][i]
        max_salaries = max_salaries + [b]
    departments.pop(0)
    max_salaries.pop(0)
    a = len(row['Department'])
    list = [] # Here I'm going to save the position of the rows i'm going to delete
    for i in range(len(departments)):
        for r in range(a):
            if (departments[i] == row['Department'][r]) & (max_salaries[i] == row['Salary'][r]):
                list = list + [r]
    list.sort()
    for i in range(len(list)): 
        row = row.drop([list[i]], axis = 0)
    new_mean = row.pivot_table(index='Department',values=['Salary'])
    return new_mean

In [13]:
new_table = delete_biggests_salaries(copy)
new_table

,Salary
Department,
HR,32.500000
IT,41.666667


In [14]:
# I have lost the Sales department because they only have 2 employees and they have the same salary, so they both have the max salarium, so they have been deleted